In [1]:
import numpy as np
import utils
import pandas as pd

In [2]:
synth_regular = utils.load_synthetic()
compas = utils.load_compas_alt()
RESPONSE = "score_factor"

In [3]:
compas.keys()

dict_keys(['train', 'test'])

In [4]:
#compas["standard"]["train"].keys()

In [5]:
#Checking that the classes don't become too much more imbalanced when data is removed.

#Synthetic data
for key, value in synth_regular.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))

Key:  test
Caucasian = 1:  0.3353353353353353 
Caucasian = 0:  0.6646646646646647
Male = 1:  0.8118118118118118 
Male = 0:  0.1881881881881882
After 30% missing: 
Caucasian = 1:  0.2821997105643994 
Caucasian = 0:  0.7178002894356006
Male = 1:  0.8219971056439942 
Male = 0:  0.17800289435600578
Key:  train
Caucasian = 1:  0.35132433783108447 
Caucasian = 0:  0.6486756621689156
Male = 1:  0.8118440779610195 
Male = 0:  0.18815592203898052
After 30% missing: 
Caucasian = 1:  0.31318252730109203 
Caucasian = 0:  0.6868174726989079
Male = 1:  0.8131825273010921 
Male = 0:  0.18681747269890794


In [6]:
a = []
b = [1,2,3]
c = [4,5,56]
a.append(b)
a.append(c)
np.mean(a, axis = 0)

array([ 2.5,  3.5, 29.5])

In [7]:
#Compas data
"""for ver in ["standard", "violent"]:
    print("Compas dataset type: ", ver)"""
for key, value in compas.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))


Key:  train
Caucasian = 1:  0.3388149939540508 
Caucasian = 0:  0.6611850060459492
Male = 1:  0.8099153567110037 
Male = 0:  0.19008464328899638
Score text:  0.44401451027811367
After 30% missing: 
Caucasian = 1:  0.32952924393723254 
Caucasian = 0:  0.6704707560627675
Male = 1:  0.8088445078459344 
Male = 0:  0.19115549215406563
Score text:  0.44401451027811367
Key:  test
Caucasian = 1:  0.3446244477172312 
Caucasian = 0:  0.6553755522827688
Male = 1:  0.8090328915071183 
Male = 0:  0.19096710849288168
Score text:  0.4491899852724595
After 30% missing: 
Caucasian = 1:  0.3361462728551336 
Caucasian = 0:  0.6638537271448663
Male = 1:  0.7883263009845288 
Male = 0:  0.21167369901547117
Score text:  0.4491899852724595


In [8]:
a = pd.DataFrame({"a":[1,2,3], "b":[3,4,5]})
b = pd.DataFrame({"a":[1,2,3], "b":[3,4,5]})
a==b

,a,b
0,True,True
1,True,True
2,True,True


In [9]:
percentiles = [p for p in range(0,20,2)]+[p for p in range(20,60, 10)]
#TODO test missing = Sensitive in case code crashes
missing=["priors_count", "two_year_recid"]# "is_Caucasian"]#, "gender_factor", "crime_factor"]
all_results = {}
RUNS = 5
for miss in missing:
    for sensitive in ["is_Caucasian"]:#, "gender_factor"]:
        #try:
        recid_results = utils.test_bench(data = "compas", pred = RESPONSE, missing = miss, sensitive=sensitive,
                        percentiles = percentiles, n_runs=RUNS)
        synth_results = utils.test_bench(data = "synthetic", pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles, n_runs=RUNS)
        #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                        #percentiles = percentiles)
        all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
        all_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
        #except:
            #print("failed at " + sensitive + miss)
        """try:
            synth_results = utils.test_bench(train = synth_regular["train"],test = synth_regular["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            recid_results = utils.test_bench(train = compas["train"],test = compas["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            #percentiles = percentiles)
            all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
            all_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
            #all_results[miss+"_"+sensitive+"_"+"v_recid"] = v_recid_results
        except Exception as e:
            print("Exception: ", e)
            print("Parameters: ", sensitive+"_"+miss) 
        """

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import json
from pathlib import Path
with open(Path("raw_data/multi_02_07.json"), 'w') as f:
            json.dump(all_results, f)